In [ ]:
import csv
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from IPython.display import SVG
from sklearn.metrics import roc_curve,roc_auc_score

In [ ]:
from utils.DataGenerator import load_data
from utils.Flops import get_flops
from utils.gradcam import GradCAM, overlay_gradCAM

In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [ ]:
# 模型參數設定
# MobileNetV3Large、 MobileNetV3Small、CustomizeLarge、 CustomizeSmall, 
backbone = 'CustomizeLarge'
# SE CBAM CA
SE_CBAM_CA = 'SE'
# Adam RMSprop CLR
LR_mode = 'Adam'

In [ ]:
# 100 Bird Species  or  325 Bird Species  or  cifar100  or  cifar10
Dataset = "325 Bird Species"
BATCH_SIZE = 1

if Dataset == "325 Bird Species":
    dir = "./Dataset/325 Bird Species/"
    input_shape = (224,224,3)
    num_classes = 325
    trainset, valset, testset = load_data(Dataset, dir, input_shape, BATCH_SIZE)
    
elif Dataset == "100 Bird Species":
    dir = "./Dataset/100 Bird Species/"
    input_shape = (224,224,3)
    num_classes = 100
    trainset, valset, testset = load_data(Dataset, dir, input_shape, BATCH_SIZE)
    
elif Dataset == "cifar100":
    input_shape = (32,32,3)
    num_classes = 100
    datagen_train, datagen_val, x_train, y_train, x_val, y_val, x_test, y_test = load_data(Dataset, '', input_shape, num_classes)
    x_train = x_train/255
    x_val = x_val/255
    x_test =x_test/255
    
elif Dataset == "cifar10":
    input_shape = (32,32,3)
    num_classes = 10
    datagen_train, datagen_val, x_train, y_train, x_val, y_val, x_test, y_test= load_data(Dataset, '', input_shape, num_classes)
    x_train = x_train/255
    x_val = x_val/255
    x_test =x_test/255

In [ ]:
if Dataset[:5] == 'cifar':
    test_generator = x_test
    Y_true = y_test
    print(np.shape(Y_true))
    y_true = np.argmax(Y_true, axis=1)
    stepNum=len(test_generator)
    print("stepNum",stepNum)
else:
    # 利用ImageDataGenerator將test樣本讀進來
    datagen_test = ImageDataGenerator(rescale= 1/255,)

    test_generator = datagen_test.flow_from_directory(dir + 'test',
                                                    target_size=(224,224),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=False
                                                    )
    y_true = test_generator.labels
    
    # test_generator = testset
    # y_true = test_generator.labels
    
    stepNum=len(y_true)
    print("stepNum",stepNum)

In [ ]:
model_dir = './weights/{0}/{1}_{2}_{3}/best_model'.format(backbone, SE_CBAM_CA, LR_mode, Dataset)
print(model_dir)
model = tf.keras.models.load_model(model_dir)

In [ ]:
# model.summary()
print("The FLOPs is:{}".format(get_flops(model)) ,flush=True )

In [ ]:
# results = model.evaluate(x_test, y_test, batch_size=256)
# print("test loss, test acc:", results)

In [ ]:
import time

mean_FPS=0
for i in range(5):
    start = time.time()

    #利用模型進行預測，記得輸入的test_generator的shuffle內建要是False
    Y_pred = model.predict(test_generator,steps=stepNum)

    end = time.time()

    # Time elapsed
    seconds = end - start
    FPS = stepNum/seconds
    if i==0:
        continue
    mean_FPS+=FPS
    # print("FPS:",FPS)
mean_FPS /= 4
print('Mean FPS:',mean_FPS)

In [ ]:
#預測值。假設有2類，則上述回傳值Y_pred目前是一個40x2(test樣本數x類別數)的矩陣
# print('Y_pred',Y_pred)#預測分數
# print('Y_pred.shape',Y_pred.shape)
# print('-'*100)

#分類結果，即每個樣本的判斷結果是什麼。例如兩個類別，y_pred就是0與1，且y_pred是(28,)(test樣本數,)的矩陣
y_pred = np.argmax(Y_pred, axis=1)

# print('y_pred=',y_pred)
# print('y_pred.shape',y_pred.shape)
# print('-'*100)

# print('y_true=',y_true)
# print('y_true.shape',y_true.shape)

In [ ]:
# Name of all classes in CIFAR-100
if Dataset=='cifar100':
    classes = ['beaver', 'dolphin', 'otter', 'seal', 'whale', 
    'aquarium' ,'fish', 'ray', 'shark', 'trout', 
    'orchids', 'poppies', 'roses', 'sunflowers', 'tulips', 
    'bottles', 'bowls', 'cans', 'cups', 'plates', 
    'apples', 'mushrooms', 'oranges', 'pears', 'sweet peppers', 
    'clock', 'computer keyboard', 'lamp', 'telephone', 'television', 'bed', 'chair', 'couch', 'table', 'wardrobe', 
    'bee', 'beetle', 'butterfly', 'caterpillar', 'cockroach', 
    'bear', 'leopard', 'lion', 'tiger', 'wolf', 
    'bridge', 'castle', 'house', 'road', 'skyscraper', 
    'cloud', 'forest', 'mountain', 'plain', 'sea', 
    'camel', 'cattle', 'chimpanzee', 'elephant', 'kangaroo', 
    'fox', 'porcupine', 'possum', 'raccoon', 'skunk', 
    'crab', 'lobster', 'snail', 'spider', 'worm', 
    'baby', 'boy', 'girl', 'man', 'woman', 
    'crocodile', 'dinosaur', 'lizard', 'snake', 'turtle', 
    'hamster', 'mouse', 'rabbit', 'shrew', 'squirrel', 
    'maple', 'oak', 'palm', 'pine', 'willow', 
    'bicycle', 'bus', 'motorcycle', 'pickup truck', 'train', 
    'lawn-mower', 'rocket', 'streetcar', 'tank', 'tractor']
elif Dataset == "cifar10":
    classes = ["airplane", "automobile", "bird", "cat",
               "deer", "dog", "frog", "horse", "ship", "truck"]
else:
    classes =[]

In [ ]:
#test_generator.classes為實際28張測試樣本的label，y_pred為預測的label
# confusion_matrixRet=confusion_matrix(y_true=y_true,y_pred= y_pred)
confusion_matrixRet=confusion_matrix(y_true = y_true, y_pred= y_pred)
print('confusion_matrix:')
print(confusion_matrixRet)

fig = plt.figure(figsize=(24,24))
ax = fig.add_subplot(211)
cax = ax.matshow(confusion_matrixRet)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + classes)
ax.set_yticklabels([''] + classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# DF = pd.DataFrame(confusion_matrixRet, columns=classes, index=classes)
DF = pd.DataFrame(confusion_matrixRet)
DF
# DF.to_csv('confusion_matrix.csv')

In [ ]:
#classification_report, 可以自動算出precision,recall與f1-score
# target_name = ['covid_19', 'lungopacity', 'normal', 'viralpneumonia']
classification_reports = classification_report(y_true = y_true,
                                              y_pred= y_pred,
                                              # target_names=target_name,
                                              output_dict=True
                                             )
# print(classification_report)#會輸出precision、recall與f1-score。

In [ ]:
Dict =  classification_reports
Col = []
for i in Dict:
    Col.append(i)

precision=[]
recall=[]
f1_score=[]
support=[]

for i in Col:
    if i == 'accuracy':
        precision.append('')
        recall.append('')
        f1_score.append(Dict[i])
        support.append('')
    else:
        precision.append(Dict[i]['precision'])
        recall.append(Dict[i]['recall'])
        f1_score.append(Dict[i]['f1-score'])
        support.append(Dict[i]['support'])  

Data = {'precision':precision,
            'recall': recall,
            'f1-score': f1_score,
            'support': support}

DF = pd.DataFrame(Data,index=Col)
# DF.to_csv("Accuracy.csv")
DF